In [1]:
import os, sys, re
import sqlite3, csv
import argparse
from tqdm.notebook import tqdm
import json
import pandas as pd
# %pip install ipywidgets

In [3]:
args = {
    "source_file": "/home/gjungwirth/data/02_data/03_analyses/statistics_for_paper/repostats.csv",
    "db": "/home/gjungwirth/data/02_data/results.db"
}

In [5]:
db = sqlite3.connect(args['db'])

db.execute('''
CREATE TABLE IF NOT EXISTS repo_stats (
    repo_id INTEGER PRIMARY KEY,
    first_commit_date TEXT NOT NULL,
    last_commit_date TEXT NOT NULL,
    no_commits INTEGER NOT NULL,
    no_authors INTEGER NOT NULL,
    FOREIGN KEY (repo_id) REFERENCES repo (repo_id)
);
''')
db.commit()

In [14]:
df = pd.read_csv(args['source_file'],
    names=['folder','first_commit','last_commit','no_commits','no_authors'])
df.info()
df.shape[0]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 124230 entries, 0 to 124229
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   folder        124230 non-null  object
 1   first_commit  123870 non-null  object
 2   last_commit   123870 non-null  object
 3   no_commits    124230 non-null  int64 
 4   no_authors    124230 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 4.7+ MB


124230

In [20]:
pbar = tqdm(total=df.shape[0])
for row in df.itertuples():
    
    raw_folder = str(row.folder).removeprefix("repos/").removeprefix("repos_part2/")
    db.execute('''
        INSERT OR IGNORE INTO repo_stats
        (repo_id, first_commit_date, last_commit_date, no_commits, no_authors)
        VALUES
        ((SELECT repo_id FROM repo WHERE folder_name = :folder_name),
        :first_commit, :last_commit, :no_commits, :no_authors);
        ''', {
            "folder_name": raw_folder,
            "first_commit": row.first_commit,
            "last_commit": row.last_commit,
            "no_commits": row.no_commits,
            "no_authors": row.no_authors
        })
    db.commit()
    pbar.update()

In [17]:
db.close()